In [253]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LassoCV, RidgeCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-pastel')

pd.set_option("display.max_rows", 501)
pd.set_option("display.max_columns", 601)

In [254]:
#Import the data
data = pd.read_csv('https://raw.githubusercontent.com/altheaxcvii/project_2/charles/datasets/train.csv')

## Data Dictionary (I collapsed it because it is very long but it's here!)

| **Column names** | **Descriptions** |
|---|---|
| resale_price | the property's sale price in Singapore dollars. This is the target variable that you're trying to predict for this challenge. |
| Tranc_YearMonth | year and month of the resale transaction e.g. 2015-02 |
| town | HDB township where the flat is located e.g. BUKIT MERAH |
| flat_type | type of the resale flat unit e.g. 3 ROOM |
| block | block number of the resale flat e.g. 454 |
| street_name | street name where the resale flat resides e.g. TAMPINES ST 42 |
| storey_range | floor level (range) of the resale flat unit e.g. 07 TO 09 |
| floor_area_sqm | floor area of the resale flat unit in square metres |
| price_per_sqft | Price per Square Foot of the unit |
| flat_model | HDB model of the resale flat e.g. Multi Generation |
| lease_commence_date | commencement year of the flat units 99-year lease |
| Tranc_Year | year of resale transaction |
| Tranc_Month | month of resale transaction |
| mid_storey | median value of storey_range |
| lower | lower value of storey_range |
| 2room_rental | 2 room rental flat |
| 3room_rental | 3 room rental flat |
| 4room_rental | 4 room rental flat |
| postal | postal code |
| other_room_rental | other room rental flat |
| upper | upper value of storey_range |
| mid | middle value of storey_range |
| full_flat_type | combination of flat_type and flat_model |
| address | combination of block and street_name |
| floor_area_sqft | floor area of the resale flat unit in square feet |
| hdb_age | number of years from lease_commence_date to present year |
| max_floor_lvl | highest floor of the resale flat |
| year_completed | year which construction was completed for resale flat |
| residential | boolean value if resale flat has residential units in the same block |
| commercial | boolean value if resale flat has commercial units in the same block |
| market_hawker | boolean value if resale flat has a market or hawker centre in the same block |
| multistorey_carpark | boolean value if resale flat has a multistorey carpark in the same block |
| precinct_pavilion | boolean value if resale flat has a pavilion in the same block |
| total_dwelling_units | total number of residential dwelling units in the resale flat |
| Latitude | Latitude of the unit |
| Longitude | Longitude of the unit |
| planning_area | planning area of the unit |
| pri_sch_nearest_distance | distance of unit to the nearest primary school |
| 1room_sold | number of 1-room residential units in the resale flat |
| 2room_sold | number of 2-room residential units in the resale flat |
| 3room_sold | number of 3-room residential units in the resale flat |
| 4room_sold | number of 4-room residential units in the resale flat |
| 5room_sold | number of 5-room residential units in the resale flat |
| exec_sold | number of executive type residential units in the resale flat block |
| pri_sch_name | name of the nearest primary school |
| vacancy | vacancy of the unit |
| pri_sch_affiliation | affiliation of primary school |
| pri_sch_latitude | latitude of primary school |
| pri_sch_longitude | longitude of primary school |
| sec_sch_nearest_dist | distance to nearest secondary school |
| sec_sch_name | name of nearest secondary school |
| cutoff_point | PSLE cutoff point of nearest secondary school |
| affiliation | if there is affiliation for the nearest secondary school |
| sec_sch_latitude | latitude of secondary school |
| sec_sch_longitude | longitude of secondary school |
| multigen_sold | number of multi-generational type residential units in the resale flat block |
| mrt_nearest_distance | distance to nearest mrt |
| mrt_name | name of nearest mrt |
| bus_interchange | if there is a bus interchange |
| mrt_interchange | if there is an mrt interchange |
| mrt_latitude | latitude of mrt |
| mrt_longitude | longitude of mrt |
| bus_stop_nearest_distance | distance to nearest bus stop |
| bus_stop_name | name of bus stop |
| bus_stop_latitude | latitude of bus stop |
| bus_stop_longitude | longitude of bus stop |
| Mall_Nearest_Distance | Distance to the nearest mall |
| Mall_Within_500m | How many malls within 500m of the unit |
| Mall_Within_1km | How many malls within 1km of the unit |
| Mall_Within_2km | How many malls within 2km of the unit |
| Hawker_Nearest_Distance | Distance to nearest Hawker Center |
| Hawker_Within_500m | How many Hawker Centers within 500m of the unit |
| Hawker_Within_1km | How many Hawker Centers within 1km of the unit |
| Hawker_Within_2km | How many Hawker Centers within 2km of the unit |
| studio_apartment_sold | number of studio apartment type residential units in the resale flat block |
| 1room_rental | number of 1-room rental residential units in the resale flat block |
| hawker_food_stalls | number of stalls at nearest hawker centre |
| hawker_market_stalls | number of market stalls at nearest hawker centre |

## Data Cleaning and Preliminary EDA

In [255]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150634 entries, 0 to 150633
Data columns (total 78 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         150634 non-null  int64  
 1   Tranc_YearMonth            150634 non-null  object 
 2   town                       150634 non-null  object 
 3   flat_type                  150634 non-null  object 
 4   block                      150634 non-null  object 
 5   street_name                150634 non-null  object 
 6   storey_range               150634 non-null  object 
 7   floor_area_sqm             150634 non-null  float64
 8   flat_model                 150634 non-null  object 
 9   lease_commence_date        150634 non-null  int64  
 10  resale_price               150634 non-null  float64
 11  Tranc_Year                 150634 non-null  int64  
 12  Tranc_Month                150634 non-null  int64  
 13  mid_storey                 15

In [256]:
#standardizing all column names to be in snake case
data.columns = data.columns.str.lower().str.replace(' ','_')

In [257]:
#dropping price_per_sqft since we cannot use this for our prediction
data.drop('price_per_sqft', axis = 1, inplace = True)

In [258]:
#any columns with only 1 unique value will not be helpful in a prediction model 
for x in data.columns:
    print(x, len(data[x].unique()))

id 150634
tranc_yearmonth 110
town 26
flat_type 7
block 2514
street_name 553
storey_range 25
floor_area_sqm 169
flat_model 20
lease_commence_date 54
resale_price 3295
tranc_year 10
tranc_month 12
mid_storey 22
lower 22
upper 23
mid 22
full_flat_type 43
address 9157
floor_area_sqft 169
hdb_age 54
max_floor_lvl 41
year_completed 59
residential 1
commercial 2
market_hawker 2
multistorey_carpark 2
precinct_pavilion 2
total_dwelling_units 320
1room_sold 2
2room_sold 99
3room_sold 255
4room_sold 200
5room_sold 133
exec_sold 103
multigen_sold 7
studio_apartment_sold 61
1room_rental 11
2room_rental 48
3room_rental 15
other_room_rental 3
postal 17483
latitude 9126
longitude 9098
planning_area 32
mall_nearest_distance 8983
mall_within_500m 7
mall_within_1km 16
mall_within_2km 42
hawker_nearest_distance 9126
hawker_within_500m 6
hawker_within_1km 10
hawker_within_2km 20
hawker_food_stalls 52
hawker_market_stalls 70
mrt_nearest_distance 9126
mrt_name 94
bus_interchange 2
mrt_interchange 2
mrt_lati

In [259]:
#Dropping residential column due to only 1 unique value and id value as it is only use for identification
data.drop(['residential', 'id'], axis = 1, inplace=True)

In [260]:
data.columns

Index(['tranc_yearmonth', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'resale_price', 'tranc_year', 'tranc_month', 'mid_storey', 'lower',
       'upper', 'mid', 'full_flat_type', 'address', 'floor_area_sqft',
       'hdb_age', 'max_floor_lvl', 'year_completed', 'commercial',
       'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'postal', 'latitude', 'longitude', 'planning_area',
       'mall_nearest_distance', 'mall_within_500m', 'mall_within_1km',
       'mall_within_2km', 'hawker_nearest_distance', 'hawker_within_500m',
       'hawker_within_1km', 'hawker_within_2km', 'hawker_food_stalls',
       'hawker_market_stalls', 'mrt_nearest_distance', 

In [261]:
data.isnull().sum().sort_values().tail(8)

max_floor_lvl                0
mall_nearest_distance      829
mall_within_2km           1940
mall_within_1km          25426
hawker_within_2km        29202
hawker_within_1km        60868
mall_within_500m         92789
hawker_within_500m       97390
dtype: int64

In [262]:
#Exploring Null Rows - looks like null value are 0 (i.e. no malls within stated distance)
data[data['mall_nearest_distance'].isnull()][['mall_nearest_distance', 'mall_within_500m', 'mall_within_1km', 'mall_within_2km']]

,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km
75,NaN,NaN,NaN,NaN
321,NaN,NaN,NaN,NaN
478,NaN,NaN,NaN,NaN
643,NaN,NaN,NaN,NaN
691,NaN,NaN,NaN,NaN
...,...,...,...,...
150296,NaN,NaN,NaN,NaN
150388,NaN,NaN,NaN,NaN
150394,NaN,NaN,NaN,NaN
150479,NaN,NaN,NaN,NaN


In [263]:
#filling null values with 0
col_with_null = data.columns[data.isnull().sum() != 0].to_list()
imputer=SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
for x in col_with_null:
    data[x]=imputer.fit_transform(data[x].values.reshape(-1,1))

In [264]:
#Since machine learning can only take numeric inputs, we will have to convert useful categorical columns to numeric form 
data.select_dtypes(include='object').columns

Index(['tranc_yearmonth', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'flat_model', 'full_flat_type', 'address', 'commercial',
       'market_hawker', 'multistorey_carpark', 'precinct_pavilion', 'postal',
       'planning_area', 'mrt_name', 'bus_stop_name', 'pri_sch_name',
       'sec_sch_name'],
      dtype='object')

In [265]:
#Checking if tranc_yearmonth is consistent with 'tranc_year', 'tranc_month'
data[['tranc_yearmonth','tranc_year', 'tranc_month']]

,tranc_yearmonth,tranc_year,tranc_month
0,2016-05,2016,5
1,2012-07,2012,7
2,2013-07,2013,7
3,2012-04,2012,4
4,2017-12,2017,12
...,...,...,...
150629,2020-09,2020,9
150630,2017-06,2017,6
150631,2020-12,2020,12
150632,2016-05,2016,5


In [266]:
#drop trancyear
data.drop('tranc_yearmonth', axis = 1, inplace=True)

In [267]:
#checking if storey_range column is consistent with 'mid_storey', 'lower', 'upper', 'mid'
data[['storey_range','mid_storey', 'lower', 'upper', 'mid']]

,storey_range,mid_storey,lower,upper,mid
0,10 TO 12,11,10,12,11
1,07 TO 09,8,7,9,8
2,13 TO 15,14,13,15,14
3,01 TO 05,3,1,5,3
4,01 TO 03,2,1,3,2
...,...,...,...,...,...
150629,04 TO 06,5,4,6,5
150630,04 TO 06,5,4,6,5
150631,10 TO 12,11,10,12,11
150632,07 TO 09,8,7,9,8


In [268]:
#Checking if mid_storey == mid columns
(data['mid_storey'] == data['mid']).value_counts()

True    150634
dtype: int64

In [269]:
#drop storey_range and mid_storey
data.drop(['storey_range','mid_storey'], axis = 1, inplace=True)

In [270]:
#checking if town == planning_area 
data[['town', 'planning_area']]

,town,planning_area
0,KALLANG/WHAMPOA,Kallang
1,BISHAN,Bishan
2,BUKIT BATOK,Bukit Batok
3,BISHAN,Bishan
4,YISHUN,Yishun
...,...,...
150629,WOODLANDS,Woodlands
150630,JURONG WEST,Jurong West
150631,BEDOK,Bedok
150632,QUEENSTOWN,Queenstown


In [271]:
data['town'] = data['town'].str.lower()
data['planning_area'] = data['planning_area'].str.lower()

In [272]:
data[['town', 'planning_area']]

,town,planning_area
0,kallang/whampoa,kallang
1,bishan,bishan
2,bukit batok,bukit batok
3,bishan,bishan
4,yishun,yishun
...,...,...
150629,woodlands,woodlands
150630,jurong west,jurong west
150631,bedok,bedok
150632,queenstown,queenstown


In [273]:
(data['town'] == data['planning_area']).value_counts()

True     144931
False      5703
dtype: int64

In [274]:
data[(data['town'] == data['planning_area']) == False][['town', 'planning_area']]

,town,planning_area
0,kallang/whampoa,kallang
32,kallang/whampoa,novena
35,kallang/whampoa,kallang
55,kallang/whampoa,kallang
88,central area,rochor
...,...,...
150507,kallang/whampoa,kallang
150510,kallang/whampoa,kallang
150511,kallang/whampoa,kallang
150524,central area,outram


In [275]:
#Planning area is more precise than town so we will drop town since most values are repeated
data.drop('town', axis = 1, inplace=True)

In [276]:
#checking if flat_type and full_flat_type is consistent
(data['flat_type'] == data['full_flat_type']).value_counts()

False    150634
dtype: int64

In [277]:
data[['flat_type', 'full_flat_type', 'flat_model']]

,flat_type,full_flat_type,flat_model
0,4 ROOM,4 ROOM Model A,Model A
1,5 ROOM,5 ROOM Improved,Improved
2,EXECUTIVE,EXECUTIVE Apartment,Apartment
3,4 ROOM,4 ROOM Model A,Model A
4,4 ROOM,4 ROOM Simplified,Simplified
...,...,...,...
150629,EXECUTIVE,EXECUTIVE Apartment,Apartment
150630,5 ROOM,5 ROOM Improved,Improved
150631,EXECUTIVE,EXECUTIVE Apartment,Apartment
150632,3 ROOM,3 ROOM Improved,Improved


In [278]:
#drop full_flat_type 
data.drop('full_flat_type', axis = 1, inplace = True)

In [279]:
#checking remaining object dtype columns and the number of values
for x in data.select_dtypes(include='object').columns:
    print(x, len(data[x].unique()))

flat_type 7
block 2514
street_name 553
flat_model 20
address 9157
commercial 2
market_hawker 2
multistorey_carpark 2
precinct_pavilion 2
postal 17483
planning_area 32
mrt_name 94
bus_stop_name 1657
pri_sch_name 177
sec_sch_name 134


In [280]:
#dropping block, street_name, address, postal and bus_stop_name as there are too many unique values 
data.drop(['block', 'street_name', 'address', 'postal', 'bus_stop_name'], axis = 1, inplace = True)

In [281]:
#Checking remaining columns
data.columns

Index(['flat_type', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'resale_price', 'tranc_year', 'tranc_month', 'lower', 'upper', 'mid',
       'floor_area_sqft', 'hdb_age', 'max_floor_lvl', 'year_completed',
       'commercial', 'market_hawker', 'multistorey_carpark',
       'precinct_pavilion', 'total_dwelling_units', '1room_sold', '2room_sold',
       '3room_sold', '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'latitude', 'longitude', 'planning_area',
       'mall_nearest_distance', 'mall_within_500m', 'mall_within_1km',
       'mall_within_2km', 'hawker_nearest_distance', 'hawker_within_500m',
       'hawker_within_1km', 'hawker_within_2km', 'hawker_food_stalls',
       'hawker_market_stalls', 'mrt_nearest_distance', 'mrt_name',
       'bus_interchange', 'mrt_interchange', 'mrt_latitude', 'mrt_longitude',
       'bus_stop_nearest_distance', 'bus_sto

In [282]:
#dropping longitude and latitude columns
data.drop(['latitude', 'longitude', 'mrt_latitude', 'mrt_longitude', 'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_latitude', 'pri_sch_longitude', 'sec_sch_latitude', 'sec_sch_longitude'], axis = 1, inplace = True)

In [283]:
#droppping floor_area_sqft becauses we have floor_area_sqm
data.drop('floor_area_sqft', axis = 1, inplace = True)

In [284]:
data.columns

Index(['flat_type', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'resale_price', 'tranc_year', 'tranc_month', 'lower', 'upper', 'mid',
       'hdb_age', 'max_floor_lvl', 'year_completed', 'commercial',
       'market_hawker', 'multistorey_carpark', 'precinct_pavilion',
       'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold',
       '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'planning_area', 'mall_nearest_distance',
       'mall_within_500m', 'mall_within_1km', 'mall_within_2km',
       'hawker_nearest_distance', 'hawker_within_500m', 'hawker_within_1km',
       'hawker_within_2km', 'hawker_food_stalls', 'hawker_market_stalls',
       'mrt_nearest_distance', 'mrt_name', 'bus_interchange',
       'mrt_interchange', 'bus_stop_nearest_distance',
       'pri_sch_nearest_distance', 'pri_sch_name', 'vacancy',
       'pri_sch_affiliation', 

In [285]:
#dropping lease_commence_date and year_completed because we have hdb_age
data.drop(['lease_commence_date','year_completed'], axis = 1, inplace = True)

In [286]:
#dropping upper and lower. mid will be used for storey information
data.drop(['upper','lower'], axis = 1, inplace = True)

In [287]:
data.select_dtypes(include='object').columns

Index(['flat_type', 'flat_model', 'commercial', 'market_hawker',
       'multistorey_carpark', 'precinct_pavilion', 'planning_area', 'mrt_name',
       'pri_sch_name', 'sec_sch_name'],
      dtype='object')

In [288]:
data.select_dtypes(include=('int64', 'float64')).columns

Index(['floor_area_sqm', 'resale_price', 'tranc_year', 'tranc_month', 'mid',
       'hdb_age', 'max_floor_lvl', 'total_dwelling_units', '1room_sold',
       '2room_sold', '3room_sold', '4room_sold', '5room_sold', 'exec_sold',
       'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental',
       'mall_nearest_distance', 'mall_within_500m', 'mall_within_1km',
       'mall_within_2km', 'hawker_nearest_distance', 'hawker_within_500m',
       'hawker_within_1km', 'hawker_within_2km', 'hawker_food_stalls',
       'hawker_market_stalls', 'mrt_nearest_distance', 'bus_interchange',
       'mrt_interchange', 'bus_stop_nearest_distance',
       'pri_sch_nearest_distance', 'vacancy', 'pri_sch_affiliation',
       'sec_sch_nearest_dist', 'cutoff_point', 'affiliation'],
      dtype='object')

### cleaning stops here

-------------------

# EDA and Features Selection
<small>Additional Guidelines for Project2:<br>
Please sign up for the project 2 kaggle competition. You will then be able to upload your predictions for the kaggle test set and get your scores.<br>
Do not use any library such as “pandas-profiling” that automate the EDA process for you. We want you to be familiar with going through EDA steps on your own.<br>
Do not include the feature “price_per_sqft” in your models as this would result in target leakage.<br>
Do not use any imputation methods that are more complex than regression or KNN imputation<br>
Do not use advanced models such as neural networks, xgboost, etc. Based on the README.md file, you would need to have 3 models (linear regression, lasso, and ridge)<br>
Do try to limit the number of features in your final model to <30 features if possible.<br>
Ensure each member has enough content to cover and time to speak

### Codes

In [289]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [290]:
# checking correlation values
data.corr()['resale_price'].sort_values().head(30)

3room_sold                  -0.410368
hdb_age                     -0.350337
2room_sold                  -0.145287
total_dwelling_units        -0.140725
mrt_nearest_distance        -0.128222
mall_nearest_distance       -0.092239
3room_rental                -0.079965
2room_rental                -0.077548
1room_sold                  -0.039434
1room_rental                -0.037993
hawker_food_stalls          -0.033581
tranc_year                  -0.019146
vacancy                     -0.016140
hawker_nearest_distance     -0.014537
pri_sch_nearest_distance    -0.011004
hawker_market_stalls        -0.009378
tranc_month                 -0.006823
affiliation                 -0.000466
bus_interchange              0.002547
hawker_within_500m           0.012637
other_room_rental            0.013049
pri_sch_affiliation          0.030931
bus_stop_nearest_distance    0.031940
cutoff_point                 0.034572
multigen_sold                0.037406
studio_apartment_sold        0.040312
mrt_intercha

In [291]:
X = data.drop(columns='resale_price')

y = data['resale_price']

X

,flat_type,floor_area_sqm,flat_model,tranc_year,tranc_month,mid,hdb_age,max_floor_lvl,commercial,market_hawker,multistorey_carpark,precinct_pavilion,total_dwelling_units,1room_sold,2room_sold,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental,planning_area,mall_nearest_distance,mall_within_500m,mall_within_1km,mall_within_2km,hawker_nearest_distance,hawker_within_500m,hawker_within_1km,hawker_within_2km,hawker_food_stalls,hawker_market_stalls,mrt_nearest_distance,mrt_name,bus_interchange,mrt_interchange,bus_stop_nearest_distance,pri_sch_nearest_distance,pri_sch_name,vacancy,pri_sch_affiliation,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation
0,4 ROOM,90.0,Model A,2016,5,11,15,25,N,N,N,N,142,0,0,0,96,46,0,0,0,0,0,0,0,kallang,1094.090418,0.0,0.0,7.0,154.753357,1.0,3.0,13.0,84,60,330.083069,Kallang,0,0,29.427395,1138.633422,Geylang Methodist School,78,1,1138.633422,Geylang Methodist School,224,0
1,5 ROOM,130.0,Improved,2012,7,8,34,9,N,N,N,N,112,0,0,0,56,56,0,0,0,0,0,0,0,bishan,866.941448,0.0,1.0,3.0,640.151925,0.0,1.0,7.0,80,77,903.659703,Bishan,1,1,58.207761,415.607357,Kuo Chuan Presbyterian Primary School,45,1,447.894399,Kuo Chuan Presbyterian Secondary School,232,0
2,EXECUTIVE,144.0,Apartment,2013,7,14,24,16,N,N,N,N,90,0,0,0,0,30,60,0,0,0,0,0,0,bukit batok,1459.579948,0.0,0.0,4.0,1762.082341,0.0,0.0,1.0,84,95,1334.251197,Bukit Batok,1,0,214.747860,498.849039,Keming Primary School,39,0,180.074558,Yusof Ishak Secondary School,188,0
3,4 ROOM,103.0,Model A,2012,4,3,29,11,Y,N,N,N,75,0,0,0,68,7,0,0,0,0,0,0,0,bishan,950.175199,0.0,1.0,4.0,726.215262,0.0,1.0,9.0,32,86,907.453484,Bishan,1,1,43.396521,389.515528,Catholic High School,20,1,389.515528,Catholic High School,253,1
4,4 ROOM,83.0,Simplified,2017,12,2,34,4,N,N,N,N,48,0,0,0,30,18,0,0,0,0,0,0,0,yishun,729.771895,0.0,1.0,2.0,1540.151439,0.0,0.0,1.0,45,0,412.343032,Khatib,0,0,129.422752,401.200584,Naval Base Primary School,74,0,312.025435,Orchid Park Secondary School,208,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150629,EXECUTIVE,161.0,Apartment,2020,9,5,26,12,Y,N,N,N,60,0,0,6,4,10,40,0,0,0,0,0,0,woodlands,585.138715,0.0,4.0,6.0,973.725973,0.0,1.0,3.0,43,0,713.675852,Woodlands,1,1,137.566352,418.983965,Evergreen Primary School,43,0,189.889876,Evergreen Secondary School,224,0
150630,5 ROOM,110.0,Improved,2017,6,5,20,16,N,N,N,N,90,0,0,0,30,60,0,0,0,0,0,0,0,jurong west,250.084466,1.0,1.0,4.0,1807.769865,0.0,0.0,1.0,72,94,239.773531,Pioneer,0,0,55.899763,136.298080,Jurong West Primary School,45,0,614.418470,Boon Lay Secondary School,188,0
150631,EXECUTIVE,144.0,Apartment,2020,12,11,28,15,N,N,N,N,56,0,0,0,0,0,56,0,0,0,0,0,0,bedok,1790.053482,0.0,0.0,2.0,1076.260436,0.0,0.0,4.0,36,75,669.284215,Kaki Bukit,0,0,143.388623,512.626814,Maha Bodhi School,43,1,556.889910,Manjusri Secondary School,188,0
150632,3 ROOM,59.0,Improved,2016,5,8,47,10,N,N,N,N,209,0,0,200,0,7,0,0,0,0,0,2,0,queenstown,587.244922,0.0,3.0,3.0,318.550377,1.0,5.0,8.0,39,63,502.030710,Commonwealth,0,0,246.171884,787.250957,New Town Primary School,82,0,832.386515,Queensway Secondary School,214,0


In [292]:
#Dummify
X = pd.get_dummies(columns=['flat_type', 'flat_model', 'commercial', 'market_hawker',
       'multistorey_carpark', 'precinct_pavilion', 'planning_area', 'mrt_name',
       'pri_sch_name', 'sec_sch_name'], data=X, drop_first=True)

In [293]:
from sklearn.preprocessing import PolynomialFeatures

# Create interaction terms
X['floor_area_sqm*hdb_age'] = X['floor_area_sqm'] * X['hdb_age']
X['total_dwelling_units*max_floor_lvl'] = X['total_dwelling_units'] * X['max_floor_lvl']

# Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X[['floor_area_sqm', 'hdb_age', 'total_dwelling_units']])

# Manually name the columns
X_poly_df = pd.DataFrame(X_poly, columns=['floor_area_sqm', 'hdb_age', 'total_dwelling_units', 
                                          'floor_area_sqm^2', 'floor_area_sqm*hdb_age', 'floor_area_sqm*total_dwelling_units', 
                                          'hdb_age^2', 'hdb_age*total_dwelling_units', 
                                          'total_dwelling_units^2'])

#Consider other features
#Floor area could be related to HDB age
X['floor_area_sqm*hdb_age'] = X['floor_area_sqm'] * X['hdb_age']


#Mall and Hawker
X['mall_within_1km*hawker_within_1km'] = X['mall_within_1km'] * X['hawker_within_1km']


#No. units dwelling is related to max floor level
X['total_dwelling_units*max_floor_lvl'] = X['total_dwelling_units'] * X['max_floor_lvl']

#MRTs are quite related to malls
X['mrt_nearest_distance*mall_within_1km'] = X['mrt_nearest_distance'] * X['mall_within_1km']

#Consider both schools as predictive values:
X['pri_sch_nearest_distance*sec_sch_nearest_dist'] = X['pri_sch_nearest_distance'] * X['sec_sch_nearest_dist']


#Vacancy x No.Dwelling units
X['vacancy*total_dwelling_units'] = X['vacancy'] * X['total_dwelling_units']


#HDB Age and max floor level
X['hdb_age*max_floor_lvl'] = X['hdb_age'] * X['max_floor_lvl']


# Concatenate the polynomial features to the original dataframe
X = pd.concat([X, X_poly_df], axis=1)



In [148]:
from sklearn.ensemble import RandomForestRegressor

# Fit a random forest model
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X, y)

# Get feature importances
importances = rf.feature_importances_

# Convert the importances into one-dimensional 1darray with corresponding df column names as axis labels
f_importances = pd.Series(importances, X.columns)

# Sort the array in descending order of the importances
f_importances.sort_values(ascending=False, inplace=True)

print(f_importances)

hawker_within_2km              1.576410e-01
floor_area_sqm                 1.197216e-01
floor_area_sqm^2               1.113619e-01
floor_area_sqm                 1.043079e-01
flat_type_EXECUTIVE            9.098432e-02
                                   ...     
mrt_name_Botanic Gardens       7.335398e-08
1room_sold                     4.910729e-08
planning_area_downtown core    3.125400e-08
mrt_name_Bendemeer             2.432268e-08
mrt_name_Bugis                 1.098108e-09
Length: 517, dtype: float64


In [161]:
print(f_importances.head(60))


hawker_within_2km                                   0.157641
floor_area_sqm                                      0.119722
floor_area_sqm^2                                    0.111362
floor_area_sqm                                      0.104308
flat_type_EXECUTIVE                                 0.090984
flat_model_New Generation                           0.057508
max_floor_lvl                                       0.047819
flat_type_4 ROOM                                    0.046715
tranc_year                                          0.028884
mrt_nearest_distance                                0.021417
mid                                                 0.016125
floor_area_sqm*hdb_age                              0.013898
hawker_food_stalls                                  0.012476
floor_area_sqm*hdb_age                              0.012440
flat_model_Model A                                  0.009382
hdb_age*max_floor_lvl                               0.007949
mall_nearest_distance   

In [162]:
#TT-Split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=5)

In [163]:
# Calculate correlations with the target
correlations = X_train.corrwith(y_train)

# Select features with a correlation above a certain threshold
features = correlations.index[correlations.abs() > 0.1]

# Subset the data
X_train_high_corr = X_train[features]
X_test_high_corr = X_test[features]

In [164]:
#Scale our features with StandardScaler to X_train and use it to transform both X_train and X_test

ss = StandardScaler()
ss.fit(X_train)
X_train_sc = ss.transform(X_train)
X_test_sc = ss.transform(X_test)

In [ ]:
#Using the features list created via random forest to select features to train
top_features = f_importances.nlargest(50).index.tolist()
feature_mask = [feature in top_features for feature in X.columns]

selector = SelectKBest(score_func=mutual_info_regression, k=50)
selector.fit(X_train_sc[:, feature_mask], y_train)

X_train_kbest = selector.transform(X_train_sc[:, feature_mask])
X_test_kbest = selector.transform(X_test_sc[:, feature_mask])


In [169]:
# Fit the Lasso model
optimal_lasso = LassoCV(n_alphas=500, cv=5)
optimal_lasso.fit(X_train_kbest, y_train)
print('Optimal Lasso alpha: ', optimal_lasso.alpha_)

# Fit the Ridge model
ridge_alphas = np.logspace(0, 5, 200)
optimal_ridge = RidgeCV(alphas=ridge_alphas, cv=5)
optimal_ridge.fit(X_train_kbest, y_train)
print('Optimal Ridge alpha: ', optimal_ridge.alpha_)

# Fit the Linear Regression model
lr = LinearRegression()
lr.fit(X_train_kbest, y_train)

# Evaluate the models
print('Lasso train score: ', optimal_lasso.score(X_train_kbest, y_train))
print('Lasso test score: ', optimal_lasso.score(X_test_kbest, y_test))
print('Ridge train score: ', optimal_ridge.score(X_train_kbest, y_train))
print('Ridge test score: ', optimal_ridge.score(X_test_kbest, y_test))
print('Linear Regression train score: ', lr.score(X_train_kbest, y_train))
print('Linear Regression test score: ', lr.score(X_test_kbest, y_test))
print("Cross Validation Score:", cross_val_score(lr, X_train, y_train, cv=20).mean())

Optimal Lasso alpha:  93.85225993899311
Optimal Ridge alpha:  1.7834308769319096
Lasso train score:  0.8570429085627163
Lasso test score:  0.8596208801980854
Ridge train score:  0.8572010419571514
Ridge test score:  0.8597307261642447
Linear Regression train score:  0.857195836018267
Linear Regression test score:  0.859712820988092
Cross Validation Score: 0.9147585684298214


In [294]:
# import test.csv and impute 0 into missing values
testdata= pd.read_csv("/Users/charles/Desktop/GAProj2/test.csv")
col_with_null_test = testdata.columns[testdata.isnull().sum() != 0].to_list()
imputer=SimpleImputer(missing_values=np.NaN, strategy='constant', fill_value=0)
for x in col_with_null_test:
    testdata[x]=imputer.fit_transform(testdata[x].values.reshape(-1,1))

In [295]:
print(testdata.columns)

Index(['id', 'Tranc_YearMonth', 'town', 'flat_type', 'block', 'street_name',
       'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date',
       'Tranc_Year', 'Tranc_Month', 'mid_storey', 'lower', 'upper', 'mid',
       'full_flat_type', 'address', 'floor_area_sqft', 'price_per_sqft',
       'hdb_age', 'max_floor_lvl', 'year_completed', 'residential',
       'commercial', 'market_hawker', 'multistorey_carpark',
       'precinct_pavilion', 'total_dwelling_units', '1room_sold', '2room_sold',
       '3room_sold', '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold',
       'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental',
       'other_room_rental', 'postal', 'Latitude', 'Longitude', 'planning_area',
       'Mall_Nearest_Distance', 'Mall_Within_500m', 'Mall_Within_1km',
       'Mall_Within_2km', 'Hawker_Nearest_Distance', 'Hawker_Within_500m',
       'Hawker_Within_1km', 'Hawker_Within_2km', 'hawker_food_stalls',
       'hawker_market_stalls', '

In [296]:
# cleaning test.csv to match the lower cases of the cleaned train.csv
testdata.columns = testdata.columns.str.lower().str.replace(' ','_')

In [297]:
#Same feature engineering as train:
# Create interaction terms
testdata['floor_area_sqm*hdb_age'] = testdata['floor_area_sqm'] * testdata['hdb_age']
testdata['total_dwelling_units*max_floor_lvl'] = testdata['total_dwelling_units'] * testdata['max_floor_lvl']

# Create polynomial features
X_poly_test = poly.transform(testdata[['floor_area_sqm', 'hdb_age', 'total_dwelling_units']])
X_poly_test_df = pd.DataFrame(X_poly_test, columns=X_poly_df.columns)

# Mall and Hawker
testdata['mall_within_1km*hawker_within_1km'] = testdata['mall_within_1km'] * testdata['hawker_within_1km']

# No. units dwelling is related to max floor level
testdata['total_dwelling_units*max_floor_lvl'] = testdata['total_dwelling_units'] * testdata['max_floor_lvl']

# MRTs are quite related to malls
testdata['mrt_nearest_distance*mall_within_1km'] = testdata['mrt_nearest_distance'] * testdata['mall_within_1km']

# Consider both schools as predictive values
testdata['pri_sch_nearest_distance*sec_sch_nearest_dist'] = testdata['pri_sch_nearest_distance'] * testdata['sec_sch_nearest_dist']

# Vacancy x No.Dwelling units
testdata['vacancy*total_dwelling_units'] = testdata['vacancy'] * testdata['total_dwelling_units']

# HDB Age and max floor level
testdata['hdb_age*max_floor_lvl'] = testdata['hdb_age'] * testdata['max_floor_lvl']

# Concatenate the polynomial features to the original test dataset
testdata = pd.concat([testdata, X_poly_test_df], axis=1)


In [298]:
# dummify test.csv
testdata_X = pd.get_dummies(columns=['flat_type', 'flat_model', 'commercial', 'market_hawker',
       'multistorey_carpark', 'precinct_pavilion', 'planning_area', 'mrt_name',
       'pri_sch_name', 'sec_sch_name'], data=testdata, drop_first=True)
testdata_X.shape

(16737, 541)

In [299]:
# Drop columns not in the feature mask
test_data_selected = testdata_X.loc[:, testdata_X.columns.isin(X.columns[feature_mask])]



In [300]:
test_data_selected.columns

Index(['floor_area_sqm', 'tranc_year', 'tranc_month', 'mid', 'hdb_age',
       'max_floor_lvl', '3room_sold', '4room_sold', '5room_sold',
       'mall_nearest_distance', 'mall_within_1km', 'mall_within_2km',
       'hawker_nearest_distance', 'hawker_within_1km', 'hawker_within_2km',
       'hawker_food_stalls', 'hawker_market_stalls', 'mrt_nearest_distance',
       'bus_stop_nearest_distance', 'pri_sch_nearest_distance', 'vacancy',
       'sec_sch_nearest_dist', 'cutoff_point', 'floor_area_sqm*hdb_age',
       'total_dwelling_units*max_floor_lvl',
       'mall_within_1km*hawker_within_1km',
       'mrt_nearest_distance*mall_within_1km',
       'pri_sch_nearest_distance*sec_sch_nearest_dist',
       'vacancy*total_dwelling_units', 'hdb_age*max_floor_lvl',
       'floor_area_sqm', 'hdb_age', 'floor_area_sqm^2',
       'floor_area_sqm*hdb_age', 'floor_area_sqm*total_dwelling_units',
       'hdb_age^2', 'hdb_age*total_dwelling_units', 'flat_type_4 ROOM',
       'flat_type_EXECUTIVE', 'flat

In [303]:
# checking columns that are in train.csv but not in test.csv
X.columns.difference(test_data_selected.columns)

Index(['1room_rental', '1room_sold', '2room_rental', '2room_sold',
       '3room_rental', 'affiliation', 'bus_interchange', 'commercial_Y',
       'exec_sold', 'flat_model_Adjoined flat',
       ...
       'sec_sch_name_Yishun Town Secondary School',
       'sec_sch_name_Yuan Ching Secondary School',
       'sec_sch_name_Yuhua Secondary School',
       'sec_sch_name_Yusof Ishak Secondary School',
       'sec_sch_name_Yuying Secondary School',
       'sec_sch_name_Zhenghua Secondary School',
       'sec_sch_name_Zhonghua Secondary School', 'studio_apartment_sold',
       'total_dwelling_units', 'total_dwelling_units^2'],
      dtype='object', length=472)

In [302]:
#Columns not in train but in test
test_data_selected.columns.difference(X.columns)

Index([], dtype='object')

In [305]:
#Dropping the difference in columns

columns_to_drop = X.columns.difference(test_data_selected.columns)
X_dropped = X.drop(columns=columns_to_drop)


In [ ]:
#StandardScaleTransform
X_test_scaled = ss.transform(test_data_selected)

#Converting back into dataframe
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=test_data_selected.columns)

#Predicting based on Lasso:
predictions = optimal_lasso.predict(X_test_scaled_df)


In [ ]:
# convert testdata_X_KBest_sc back into a data frame
testdata_X_KBest_sc_df = pd.DataFrame(testdata_X_KBest_sc, columns = testdata_X_KBest.columns)
testdata_X_KBest_sc_df.head()

,floor_area_sqm,mid,hdb_age,max_floor_lvl,total_dwelling_units,2room_sold,3room_sold,5room_sold,exec_sold,hawker_within_2km,mrt_nearest_distance,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_model_Apartment,flat_model_DBSS,flat_model_Maisonette,flat_model_New Generation,flat_model_Simplified,commercial_Y,planning_area_bishan,planning_area_bukit merah,planning_area_yishun,mrt_name_Bishan,pri_sch_name_Cantonment Primary School,pri_sch_name_Catholic High School,pri_sch_name_Zhangde Primary School,sec_sch_name_Outram Secondary School,sec_sch_name_Queenstown Secondary School
0,-0.528647,-0.050219,0.438865,-0.510938,0.124289,-0.155017,-0.572020,0.425095,-0.301045,-0.700542,-0.185970,-0.119334,-0.593888,-0.559661,-0.285816,-0.200366,-0.095096,-0.166789,-0.424156,4.380569,2.010290,-0.141822,-0.201814,3.777696,-0.111081,-0.080964,-0.096051,-0.113805,-0.103089,-0.073528
1,0.616688,-0.595299,-1.286615,-0.184461,-1.235164,-0.155017,-0.572020,-0.028338,-0.301045,-0.947358,-1.386416,-0.119334,-0.593888,1.786795,-0.285816,-0.200366,-0.095096,-0.166789,-0.424156,-0.228281,-0.497441,-0.141822,-0.201814,-0.264712,-0.111081,-0.080964,-0.096051,-0.113805,-0.103089,-0.073528
2,-1.183124,-0.050219,1.014025,-0.510938,1.604199,-0.155017,2.202729,-0.753830,-0.247447,1.767614,-0.168422,-0.119334,1.683818,-0.559661,-0.285816,-0.200366,-0.095096,-0.166789,2.357623,-0.228281,-0.497441,-0.141822,-0.201814,-0.264712,-0.111081,-0.080964,-0.096051,-0.113805,-0.103089,-0.073528
3,0.003116,-1.140380,1.096191,-0.184461,-0.357542,-0.155017,-0.572020,-0.784059,-0.301045,-0.453727,0.813377,-0.119334,-0.593888,-0.559661,-0.285816,-0.200366,-0.095096,-0.166789,2.357623,-0.228281,-0.497441,-0.141822,-0.201814,-0.264712,-0.111081,-0.080964,-0.096051,-0.113805,-0.103089,-0.073528
4,0.248545,1.585022,0.603196,1.611162,0.330788,-0.155017,-0.572020,2.117911,-0.301045,-0.453727,0.513423,-0.119334,-0.593888,-0.559661,-0.285816,-0.200366,-0.095096,-0.166789,-0.424156,-0.228281,-0.497441,-0.141822,-0.201814,-0.264712,-0.111081,-0.080964,-0.096051,-0.113805,-0.103089,-0.073528


In [ ]:
# make predictions
test_pred = ridge.predict(testdata_X_KBest_sc_df)
test_pred.shape

(16737,)

In [ ]:
# make a new dataframe with id of test.csv
test_data_export = pd.read_csv("https://raw.githubusercontent.com/altheaxcvii/project_2/Jackson's/datasets/test.csv")
test_data_export2 = test_data_export['id'].to_frame()
test_data_export2

,id
0,114982
1,95653
2,40303
3,109506
4,100149
...,...
16732,23347
16733,54003
16734,128921
16735,69352


In [ ]:
# add a new series for predictions to the id dataframe
test_data_export2['Predicted'] = test_pred

In [ ]:
# export csv for Kaggle
test_data_export2.to_csv('2test_KBest30_testsize20_testcsvSSTransformed_LabRegularizationStyle.csv')